In [ ]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
import tqdm
from datetime import datetime

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [ ]:
from utils import solr
from utils.logger import logger as log

log.info('START')

ROWS = solr.select('experiment', {'params':{'q':'*:*', 'rows':0}})['response']['numFound']

print(ROWS)

In [ ]:
data = solr.select('experiment', 
{
    'params':
    {
        'q':'*:*',
        'wt':'csv',
        'rows': 0,
        'facet':''
    }
})

fl = pd.Series(data.decode('utf-8').split(','))

fl = ','.join(fl.sort_values().str.strip().tolist())

fl.split(',')

In [ ]:
q = f'''
specimen_id:[* TO *]  AND 
sex:[* TO *]  AND observation_type:unidimensional AND life_stage_name:[* TO *] AND

(
    (procedure_name:"Body Weight" AND parameter_name:"Body weight") OR
    (procedure_name:"Body Weight" AND parameter_name:"Body Weight")
)
'''
     
fl = 'id, specimen_id, gene_symbol, gene_accession_id, allele_symbol, allele_accession_id, zygosity, biological_sample_group, strain_name, sex, observation_type, life_stage_name, age_in_days, data_point, _version_'

data = solr.select('experiment', 
{
    'params':
    {
        'q':q,
        'rows': ROWS,
        'fl':fl
    }
})

log.info('Create dataframe.')

df = pd.DataFrame(data['response']['docs'])

log.info('Write to file.')

df.to_pickle('gene.pkl')